# Figure 4: Volume comparison between M22 and F19

In [ ]:
import os 
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

Notice there are basins with glacier_area > 0 and glacier_volume == 0 because the volume assignment is based on the glacier's terminus position

In [ ]:
# data at the catchment scale
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")
basins = basins[["basin_id", "geometry"]].set_index("basin_id")

data = pd.read_csv("MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id")
data['basin_name']= data['basin_name'].replace({'Santa Cruz': 'Santa Cruz                          '})
basins = pd.concat([basins, data], axis=1)
basins["diff"] = ((basins['vol_M22']/basins['vol_F19'])-1)*100

# data at the catchment scale
data = basins[["basin_zone","vol_M22","vol_F19"]].groupby("basin_zone").sum()
data["delta_p"] = (data.vol_M22 / data.vol_F19) - 1
data["delta_abs"] = (data.vol_M22 - data.vol_F19).astype("int")
delta_abs  = [str(x)+' km<sup>3</sup>' for x in data["delta_abs"]]

## Plot elements 

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -72.8, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

## Plot

In [ ]:
fig = make_subplots(rows=2, cols=3, column_titles = ["Normalized thickness in F19", "Volume difference per catchment rel. to F19 ", "Volume difference per zone rel. to F19 "], 
                    horizontal_spacing = 0.01, specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}, {"type": "xy"}], 
                                                      [None,                                 None,                                  None]], row_heights = [2,-0.18])

cl = px.colors.colorbrewer.RdYlBu
cs = px.colors.colorbrewer.GnBu

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)

# Figure a -------------------------------------------------------------------------------------------------------------------

colorbar_config_a = dict(title='Thickness<br>(log m)', len=0.4, x=0.24, y= 0.79, thickness=20, tickwidth = 1, 
                         ticktext = ["10<sup>-1", "10<sup>0", "10<sup>1", "10<sup>2", "10<sup>3"], tickvals =  [-1,0,1,2, 3])
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = np.log10((basins['vol_F19'] / basins['basin_area'])*1000), 
                            colorscale = [cs[1], cs[4], cs[8]], marker_line_color ='rgba(255,255,255, 0.6)', marker_line_width=0.5, 
                            zmin = -1, zmax = 3, colorbar = colorbar_config_a), row=1, col=1)


# Figure b -------------------------------------------------------------------------------------------------------------------

colorbar_config_b = dict(title='Difference', len=0.4, x=0.57, y = 0.79, ticktext = ["< -50 %", "-25 %", "0 %", "25 %", "> 50 %"], tickvals =  [-50,-25,0,25,50], thickness=20, tickwidth = 1)
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins["diff"], 
                            colorscale=[cl[2],"#ffe9ba", cl[9]], marker_line_color='rgba(255,255,255, 0.6)', marker_line_width=0.5, 
                           zmax = 50, zmin = -50, colorbar = colorbar_config_b), row=1, col=2)
## for Fig a) and b)
for x in range(1,3):
    
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.8, color = 'black'), opacity = 0.7, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.7)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['basin_name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

# Figure c -------------------------------------------------------------------------------------------------------------------
fig.add_trace(go.Bar(y=data.index, x=data['delta_p'], orientation='h', text= delta_abs, insidetextfont = dict(size=15),
                     marker= dict(color = data['delta_p'], colorscale = [cl[3], "#ffe9ba", cl[8]], showscale=False)), row=1, col=3)
fig.update_traces(showlegend=False, opacity=0.9, textfont_size=15, textangle=0, textposition="outside", marker_line_color='rgba(0,0,0,0.6)', marker_line_width=1, cliponaxis=False, row=1, col=3)
fig.update_xaxes(title = "Difference per zone (%)", title_standoff = 0, zeroline = True, zerolinecolor = 'rgba(0,0,0,0.5)', zerolinewidth = 1, range = [-0.19,0.19], dtick = 0.1, tickformat = ',.0%', row=1, col=3)
fig.update_yaxes(side = "right", categoryorder='array', categoryarray= ['CDI', 'GCN', 'SPI-S', 'SPI-C', 'SPI-N', 'NPI-W','NPI-E','PCA','PPY'], row=1, col=3)
fig.update_traces(textposition='auto', row=1, col=3)
fig.add_annotation(text="Absolute difference", font=dict(size=14), x=-0.13, y=0.25, ax=30, showarrow=True, arrowhead=0, arrowcolor="#636363", row=1, col=3)

# Layout -------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)

fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)

fig.add_annotation(text="(a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="(b)", font=dict(size=16), x=0.36, y=0.995,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="(c)", font=dict(size=16), x=0.70, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

fig.update_layout(autosize = False, width = 1150, height = 700, template = "seaborn", margin = dict(l=10, r=5, b=5, t=30))
fig.show()

#fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_4_thickness.png", scale=4)

## Main text

In [ ]:
"The hydrological zones comprising the SPI have an ice volume of {:.0f} km3, representing {:.1f}% of the study area".format(
    data.vol_F19.iloc[6:].sum(), 
    data.vol_F19.iloc[6:].sum()*100/data.vol_F19.sum())

In [ ]:
"Conversely, the PPY, PCA, GCN and CDI zones accounted for only {:.1f}% of the total ice volume".format(
    (data.vol_F19.iloc[0:2].sum() + data.vol_F19.iloc[4:6].sum())*100/data.vol_F19.sum())

In [ ]:
"The {:.1f}% of the study domain had a normalized thickness (ice volume divided by catchment area) of less than 1.0 m".format(
basins[(basins['vol_F19']/basins['basin_area'])*1000 < 1].basin_area.sum()*100/basins.basin_area.sum())

In [ ]:
"Based on RGI6, the M22 dataset showed more ice volume than the F19 dataset in {:.1f}% of the total glacier area".format(
    basins[basins['vol_M22'] > basins['vol_F19']].area_RGI6.sum()*100/basins.area_RGI6.sum())

In [ ]:
"Overall difference of {:.1f}%".format( (basins.vol_M22.sum() - basins.vol_F19.sum())*100/basins.vol_F19.sum())

In [ ]:
"The glacier area-weighted means of relative differences was {:.1f}%".format(
    (basins["diff"] * basins.area_RGI6).sum() / basins.area_RGI6.sum()) 